In [1]:
import json
import pandas as pd
import altair as alt
import seaborn as sns
import numpy as np

In [2]:
with open("budget_by_function.json", "r") as f:

    data = json.load(f)  

In [100]:
df_list = [
    pd.DataFrame(data[qtr])
    .assign(quarter=qtr)
    .pivot(index='quarter', columns='name', values='amount')
    for qtr in data.keys()
]

# Budget by quarter
budget_by_qtr = pd.concat(df_list)
budget_qtr_ = budget_by_qtr.reset_index()
budget_qtr_plot = budget_qtr_.melt(id_vars=['quarter'], var_name='functions', value_name='value')

# Budget by year
budget_qtr_['year'] = budget_qtr_.quarter.str[:4]
budget_by_year = budget_qtr_.iloc[:, 1:].groupby('year').sum()
budget_year_plot = budget_by_year.reset_index().melt(id_vars=['year'], var_name='functions', value_name='value')

# Total Budget
total_spending_by_year = budget_by_year.sum(axis =1).reset_index()
total_spending_by_year.columns = ['year','total_spending']

## Plot Bar chart for trend in total spending

In [61]:
alt.Chart(total_spending_by_year).mark_bar(color='#003262',cornerRadiusTopLeft=5, cornerRadiusTopRight=5).encode(
    x=alt.X('year:O', title='Fiscal Year'),
    y=alt.Y('total_spending:Q', title='Total Spending', axis=alt.Axis(format='.1f', labelExpr="datum.value/ 1E12 + 'T'")),
    tooltip = ['year','total_spending']
).properties(
    width=600,
    height=500,
    title='Total Spending'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    grid=False
).configure_title(
    fontSize=18,
    fontWeight='bold'
).configure_view(
    strokeWidth=0  # Removes border around chart for a cleaner look
)

alt.Chart(...)

## Click into year 2021

In [114]:
spending_2021 = pd.DataFrame(budget_by_year.diff().loc['2021']).sort_values(by='2021').reset_index().reset_index().rename(columns={'index': 'order'})
spending_2021['base'] = spending_2021['2021'].cumsum().shift(1).fillna(0)
spending_2021['end'] = spending_2021['base'] + spending_2021['2021']
spending_2021['change_type'] = np.where(spending_2021['2021'] >= 0, 'Increase', 'Decrease')

waterfall = alt.Chart(spending_2021).mark_bar(
    cornerRadiusTopLeft=5,
    cornerRadiusTopRight=5
).encode(
    x=alt.X('name:N', title='Category', 
           axis=alt.Axis(labelAngle=-45),
           sort=alt.EncodingSortField(field='order', order='ascending')),
    y=alt.Y('end:Q', title='Cumulative Spending',
           axis=alt.Axis(format='.1f', labelExpr="datum.value / 1E9 + 'B'")),
    y2='base:Q',
    color=alt.Color('change_type:N',
                  scale=alt.Scale(domain=['Increase', 'Decrease'],
                                 range=['#4ae27d', '#e24a4a']),
                  legend=None),
    tooltip=[
        alt.Tooltip('name:N', title='Category'),
        alt.Tooltip('2021:Q', title='Change', format='$.1f'),
        alt.Tooltip('end:Q', title='Total', format='$.1f')
    ]
).transform_window(
    sort=[{'field': 'order', 'order': 'ascending'}],
    next_name='lead(name)',
    next_end='lead(end)'
).properties(
    width=600,
    height=500,
    title='Spending Changes from Previous Year (Waterfall Chart)'
)

# Add connector lines (FIXED VERSION)
connectors = alt.Chart(spending_2021).mark_rule(
    color='gray', 
    size=1
).encode(
    x=alt.X('name:N',sort=alt.EncodingSortField(field='order', order='ascending')),
    x2='name:N',
    y='end:Q',
    y2=alt.Y2('end_lead:Q')  # Use the precomputed lead value
).transform_window(
    end_lead='lead(end)',
    sort=[{'field': 'order', 'order': 'ascending'}]
)

(waterfall + connectors).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    grid=False
).configure_title(
    fontSize=18,
    fontWeight='bold'
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

In [145]:
spending_2021 = pd.DataFrame(budget_by_year.diff().loc['2021']).sort_values(by='2021').reset_index().reset_index().rename(columns={'index': 'order'})
spending_2021['base'] = spending_2021['2021'].cumsum().shift(1).fillna(0)
spending_2021['end'] = spending_2021['base'] + spending_2021['2021']
spending_2021['change_type'] = np.where(spending_2021['2021'] >= 0, 'Increase', 'Decrease')
pct_change_df = pd.DataFrame(budget_by_year.pct_change().loc['2021']).rename(columns={'2021': 'pct_change'}).reset_index()
spending_2021 = spending_2021.merge(pct_change_df, how='inner',on='name')

waterfall = alt.Chart(spending_2021).mark_bar(
    cornerRadiusTopLeft=5,
    cornerRadiusTopRight=5
).encode(
    x=alt.X('name:N', title='Category', 
           axis=alt.Axis(labelAngle=-45),
           sort=alt.EncodingSortField(field='order', order='ascending')),
    y=alt.Y('end:Q', title='Absolute Change in Spending',
           axis=alt.Axis(format='.1f', labelExpr="datum.value / 1E9 + 'B'")),
    y2='base:Q',
    color=alt.Color('change_type:N',
                  scale=alt.Scale(domain=['Increase', 'Decrease'],
                                 range=['#4ae27d', '#e24a4a']),
                  legend=None),
    tooltip=[
        alt.Tooltip('name:N', title='Category'),
        alt.Tooltip('2021:Q', title='Change', format='$.1f'),
        alt.Tooltip('end:Q', title='Total', format='$.1f')
    ]
).transform_window(
    sort=[{'field': 'order', 'order': 'ascending'}],
    next_name='lead(name)',
    next_end='lead(end)'
).properties(
    width=600,
    height=500,
    title='2021 Spending Changes from Previous Year'
)

# Add connector lines (FIXED VERSION)
connectors = alt.Chart(spending_2021).mark_rule(
    color='gray', 
    size=1
).encode(
    x=alt.X('name:N',sort=alt.EncodingSortField(field='order', order='ascending')),
    x2='name:N',
    y='end:Q',
    y2=alt.Y2('end_lead:Q')  # Use the precomputed lead value
).transform_window(
    end_lead='lead(end)',
    sort=[{'field': 'order', 'order': 'ascending'}]
)

points = alt.Chart(spending_2021).mark_point(
    filled=True,
    size=100,
    color='#003262'
).encode(
    x=alt.X('name:N', sort=alt.EncodingSortField(field='order', order='ascending')),
    y='pct_change:Q',
    shape=alt.Shape('change_type:N',
                  scale=alt.Scale(domain=['Increase', 'Decrease'],
                                 range=['triangle-up', 'triangle-down']),
                  legend=None),
    tooltip=[alt.Tooltip('pct_change:Q', format='.1%')]
)

# Combine all elements
final_chart = alt.layer(
    waterfall + connectors, 
    points
).resolve_scale(
    x='shared',
    y='independent'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    grid=False
).configure_title(
    fontSize=18,
    fontWeight='bold'
).configure_view(
    strokeWidth=0
)

final_chart

alt.LayerChart(...)

## Plot Bar chart from 2021-2024

In [17]:
top_10_by_year = budget_by_year.loc[:, list(budget_by_year.loc['2024'].sort_values(ascending=False)[:10].index)].iloc[-4:,:]
top_10_by_year_plot = top_10_by_year.reset_index().melt(id_vars=['year'], var_name='functions', value_name='value')

In [18]:
top_10_cum_growth = (top_10_by_year.pct_change() + 1).cumprod()
top_10_cum_growth.loc['2021'] = 1
order = list(top_10_cum_growth.iloc[-1].sort_values().index)

top_10_by_year_plot = top_10_cum_growth.reset_index().melt(id_vars=['year'], var_name='functions', value_name='value')
# cum_growth.loc['2024'].sort_values()
alt.Chart(top_10_by_year_plot).mark_bar().encode(
    x=alt.X('year:O'),
    y=alt.Y('value:Q', title='cumulative growth from 2021'),
    color=alt.Color('functions:N', sort=order, scale=alt.Scale(scheme='set3')),
    xOffset=alt.XOffset('functions:N',  sort=order),  # This creates the side-by-side bars
    tooltip = ['functions','value']
).properties(
    title='Top 10 Budget Categories: Cumulative Growth Since 2021'  # Add your title here
)

alt.Chart(...)

## Plot line chart

In [26]:
# top_10_cum_growth = (budget_by_year.pct_change() + 1).cumprod()
top_10_by_year = budget_by_year.loc[:, list(budget_by_year.loc['2024'].sort_values(ascending=False)[:10].index)]
top_10_cum_growth = (top_10_by_year.pct_change() + 1).cumprod()
top_10_cum_growth.loc['2018'] = 1
order = list(top_10_cum_growth.iloc[-1].sort_values().index)

top_10_by_year_plot = top_10_cum_growth.reset_index().melt(id_vars=['year'], var_name='functions', value_name='value')

line = alt.Chart(top_10_by_year_plot).mark_line().encode(
    x=alt.X('year:O'),
    y=alt.Y('value:Q', title='cumulative growth since 2018'),
    color=alt.Color('functions:N', sort=order, scale=alt.Scale(scheme='set3')),
    tooltip=['year:O','functions:N','value:Q']  # Add formatting
).transform_filter(  # Filter out 'housing' category
    alt.datum.functions != 'Commerce and Housing Credit'
).properties(
    title='Top 10 Budget Categories: Cumulative Growth Since 2018',
    width=700,
    height=400,
)

interactive_line = line.interactive()  # Enable zoom/pan

# Add hover highlighting
highlight = alt.selection_point(
    on='mouseover',
    fields=['functions'],
    empty=False
)

# Highlight lines on hover
highlighted_lines = line.encode(
    strokeWidth=alt.condition(highlight, alt.value(5), alt.value(2))
).add_params(highlight)

highlighted_lines

alt.Chart(...)

## Interactive Indexed Line Chart

In [175]:
budget_year_plot['indexed'] = budget_year_plot.groupby('functions')['value'].transform(
    lambda x: (x / x.iloc[0])
)

# Basic indexed line chart
line = alt.Chart(budget_year_plot).mark_line().encode(
    x=alt.X('year:T', title='year'),
    y=alt.Y('indexed:Q', title='cumulative growth'),
    color=alt.Color('functions:N'),
    tooltip=['functions:N', 'indexed:Q', 'value:Q']
).properties(
    width=600,
    height=400,
    title='Indexed Line Chart (Normalized to Starting Value)'
)

interactive_line = line.interactive()  # Enable zoom/pan

# Add hover highlighting
highlight = alt.selection_point(
    on='mouseover',
    fields=['functions'],
    empty=False
)

# Highlight lines on hover
highlighted_lines = line.encode(
    strokeWidth=alt.condition(highlight, alt.value(3), alt.value(1))
).add_params(highlight)

# Combine all layers
final_chart = (interactive_line + highlighted_lines)
final_chart

alt.LayerChart(...)

In [13]:
# # 1. Define Hover Selection
# hover = alt.selection_single(
#     on='mouseover',
#     fields=['year'],
#     nearest=True,
#     empty='none',
#     name='hover'
# )

# # 2. Compute Indexed Values Dynamically (No LookupData)
# chart = alt.Chart(budget_year_plot).mark_line().encode(
#     x='year:T',
#     y=alt.Y('indexed:Q', title='Indexed Value (Base=100)'),
#     color='functions:N',
#     tooltip=['year:T', 'functions:N', 'indexed:Q', 'value:Q']
# ).transform_calculate(
#     # Calculate base value for each category at hovered date
#     base_value=f"datum.year === hover.year ? datum.value : 0"
# ).transform_joinaggregate(
#     total_base='sum(base_value)',
#     groupby=['functions']
# ).transform_calculate(
#     indexed='datum.value / datum.total_base * 100'
# ).add_params(hover).interactive()

# # 3. Add Vertical Rule for Hover Feedback
# rule = alt.Chart(budget_year_plot).mark_rule(color='gray').encode(
#     x='year:T'
# ).transform_filter(hover)

# # Combine and display
# final_chart = (chart + rule).properties(
#     title='Hover to Rebase Lines to Selected Date'
# )
# final_chart

### Hypotheses (the questions I was curious about before looking at the data)
- Did National Defense contribute the most to government spending growth?<br>
  To what extent has National Defense contributed to overall government spending growth compared to other major budget categories?
- Is the growth in spending primarily due to inefficiency?<br>
  How much of the growth in federal spending is attributable to administrative costs, waste, or redundancy, versus increased program benefits and new initiatives?
- Did government receipts change significantly from 2021–2024 due to tax policy shifts?<br>
  (this may need control factors (e.g., economic cycles, COVID recovery))
- How do climate change/trade policies manifest in budget allocations?<br>
  How have budget allocations for energy, environmental programs, and trade-related subsidies changed under different administrations?


In [4]:
# Sort budget functions by the amount in 2024
sort_by_2024 = list(budget_year_plot[budget_year_plot.year == '2024'].sort_values(by='value', ascending=False)['functions'])

In [5]:
alt.Chart(budget_year_plot).mark_area().encode(
    x=alt.X('year(year):T', title='Year'),
    y=alt.Y('value:Q', axis=alt.Axis(format='~s')),
    color=alt.Color('functions:N', 
                   scale=alt.Scale(domain=sort_by_2024),
                   sort=sort_by_2024)  # This ensures both color legend and stacking order match
).properties(
    width=600,
    height=400
)

alt.Chart(...)